In [1]:
import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp
from genpen.utils import Paper
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
import vpype
from skimage import io
from pathlib import Path

from sklearn.preprocessing import minmax_scale
from skimage import feature
from skimage import exposure

from skimage import filters
from skimage.color import rgb2gray
from skimage.transform import rescale, resize, downscale_local_mean
from skimage.morphology import disk

In [76]:
def local_angle(dx, dy):
    """Calculate the angles between horizontal and vertical operators."""
    return np.mod(np.arctan2(dy, dx), np.pi)

In [114]:
image_path =  '/mnt/c/code/side/blender/renders/coda339_render3.png'
filename = '0305_coda339_hatch.svg'
paper_size:str = '14x11 inches'
border:float=20  # mm
image_rescale_factor:float=0.1
smooth_disk_size:int=2
hist_clip_limit=0.1
hist_nbins=32
intensity_min=0.
intensity_max=1.
hatch_spacing_min=0.3  # mm
hatch_spacing_max=1.2  # mm
pixel_width=1  # mm
pixel_height=1  # mm
angle_jitter='ss.norm(scale=25).rvs'  # degrees
# angle_jitter='0'
pixel_rotation='0'  # degrees
merge_tolerances=[0.1, 0.2, 0.3]  # mm
simplify_tolerances=[0.05, 0.1, ]  # mm
savedir='/mnt/c/code/side/plotter_images/oned_outputs'

In [115]:
# make page
paper = Paper(paper_size)
drawbox = paper.get_drawbox(border)

# load
img =  rgb2gray(io.imread(Path(image_path)))
img_rescale = rescale(img, image_rescale_factor)

# 
img_renorm = exposure.equalize_adapthist(img_rescale, clip_limit=hist_clip_limit, nbins=hist_nbins)

# calc dominant angle
selem = disk(smooth_disk_size)
filt_img = filters.rank.mean(img_renorm, selem)
angle_farid = local_angle(filters.farid_h(filt_img), filters.farid_v(filt_img))

# make pixel polys
prms = []
for y, row in tqdm(enumerate(img_renorm)):
    for x, intensity in enumerate(row):

        p = gp.centered_box(Point(x, y), width=pixel_width, height=pixel_height)
        a = np.degrees(angle_farid[y, x])
        prm = {
            'geometry':p,
            'x':x,
            'y':y,
            'raw_pixel_width':pixel_width,
            'raw_pixel_height':pixel_height,
            'intensity': intensity,
            'angle':a,
            'group': 'raw_hatch_pixel',

        }
        prms.append(prm)
raw_hatch_pixels = geopandas.GeoDataFrame(prms)

<ipython-input-115-4fe1936fb6c9>:6: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  img =  rgb2gray(io.imread(Path(image_path)))
/home/anaka/anaconda3/envs/genpen/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank filters. Convert manually using skimage.util.img_as_ubyte to silence this warning.
  exec(code_obj, self.user_global_ns, self.user_ns)
108it [00:00, 113.81it/s]


In [116]:
#  rescale polys to fit in drawbox
bbox = box(*raw_hatch_pixels.total_bounds)
_, transform = gp.make_like(bbox, drawbox, return_transform=True)
A = gp.AffineMatrix(**transform)
scaled_hatch_pixels = raw_hatch_pixels.copy()
scaled_hatch_pixels['geometry'] = scaled_hatch_pixels.affine_transform(A.A_flat)

In [117]:
example_height = gp.get_height(scaled_hatch_pixels.loc[0, 'geometry'])
example_width = gp.get_width(scaled_hatch_pixels.loc[0, 'geometry'])
print(f'pixel size = {example_width:.2}x{example_height:.2}mm')
scaled_hatch_pixels['scaled_pixel_height'] = example_height
scaled_hatch_pixels['scaled_pixel_width'] = example_width

pixel size = 1.6x1.6mm


In [118]:
scaled_hatch_pixels['scaled_pixel_min_dim'] = scaled_hatch_pixels.loc[:, ['scaled_pixel_height', 'scaled_pixel_width']].min(axis=1)

In [119]:
# distributions etc
angle_jitter_gen = gp.make_callable(eval(angle_jitter))
pixel_rotation_gen = gp.make_callable(eval(pixel_rotation))


scaled_hatch_pixels['angle_jitter'] = angle_jitter_gen(len(scaled_hatch_pixels))
scaled_hatch_pixels['hatch_angle'] = scaled_hatch_pixels['angle'] + scaled_hatch_pixels['angle_jitter']
scaled_hatch_pixels['pixel_rotation'] = pixel_rotation_gen(len(scaled_hatch_pixels))

In [101]:
intensity_min = 0
intensity_max = 1

In [102]:
hatch_spacing_min = 0.25

In [120]:
hatch_spacing_max = 1.5

In [121]:
spacing_func = functools.partial(np.interp, xp=[intensity_min, intensity_max], fp=[hatch_spacing_max, hatch_spacing_min, ])
scaled_hatch_pixels['spacing'] = spacing_func(1 - scaled_hatch_pixels['intensity'])

In [122]:
filt_scaled_hatch_pixels = scaled_hatch_pixels.query('spacing < scaled_pixel_min_dim')

In [123]:
new_rows = []
for i, row in tqdm(filt_scaled_hatch_pixels.iterrows(), total=len(filt_scaled_hatch_pixels)):
    r = row.copy()

    p = r['geometry']
    if abs(r['pixel_rotation']) > np.finfo(float).eps:
        p = sa.rotate(p, r['pixel_rotation'])
    f = gp.hatchbox(p, spacing=r['spacing'], angle=r['hatch_angle'])
    r['geometry'] = f
    new_rows.append(r)

fills = geopandas.GeoDataFrame(new_rows)
fills = fills[fills.length > 0]
fill_layer = gp.merge_LineStrings(fills.geometry)

100%|██████████| 20736/20736 [00:08<00:00, 2566.17it/s]


In [128]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.stroke(1)
sk.geometry(fill_layer)

In [129]:
sk.display()

<?xml version="1.0" encoding="utf-8" ?>
 
 
 
 
 image/svg+xml 
 
 2021-03-13T22:27:43.649058 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

In [130]:
merge_tolerances=[0.1, 0.2, 0.3, 0.6, 1.2, 2.4]  # mm
simplify_tolerances=[0.05, 0.1,]  # mm

In [131]:
for tolerance in merge_tolerances:
    sk.vpype(f'linemerge --tolerance {tolerance}mm')
            
for tolerance in simplify_tolerances:
    sk.vpype(f'linesimplify --tolerance {tolerance}mm')
sk.display()

<?xml version="1.0" encoding="utf-8" ?>
 
 
 
 
 image/svg+xml 
 
 2021-03-13T22:28:13.180982 
 
 
 
 
 
 <polyline points="199.0935,198.7323 194.379,204.9449 193.6299,201.0034 187.4173,202.6738 187.4173,199.9795 179.9376,204.9449 176.2593,211.1575 174.9921,207.9581 168.7795,208.447 162.5669,207.6554 156.3543,208.115 156.3543,209.6923 150.1417,206.41 150.1417,201.4902 143.9291,202.187 143.9291,193.5281 137.7165,197.7238 137.7165,194.6126 131.5039,196.6394 131.5039,195.4453 119.0787,195.8948 119.0787,194.6853 112.8661,196.5667 112.8661,194.9106 106.6535,196.3414 106.6535,195.485 100.4409,195.767 100.4409,195.3098 94.2283,195.9422 94.2283,194.9846 88.0157,196.2673 88.0157,194.5464 81.8031,196.7055 81.8031,194.5893 75.5906,196.6626 81.8031,200.3719 75.5906,203.3053 75.5906,207.0673 81.8031,209.0351 84.1778,211.1575 85.6411,204.9449 81.8031,204.4008 88.0157,199.2764"/>
 <polyline points="268.1811,192.9943 261.9685,198.2576 261.9685,193.0182 255.7559,198.2338 252.8518,198.7323 252.4474,192.5197 249.5433,193.3415 243.3307,197.9105 243.3307,194.2205 237.1181,197.0315 236.6088,192.5197 231.4149,198.7323 230.9055,197.4208 224.6929,193.8311 224.6929,196.3016 218.4803,194.9503 218.2136,192.5197 212.5344,198.7323 212.2677,193.5176 206.0551,197.7344 206.0551,195.9867 199.8425,195.2653 193.6299,195.6763 193.6299,194.9615 187.4173,196.2905 186.7388,192.5197 181.8832,198.7323 180.3116,198.7323 175.8852,204.9449 174.9921,202.4862 168.7795,201.191 168.7795,201.6622 162.5669,202.3386 156.3543,201.3385 156.3543,194.7133 150.1417,196.5387 150.1417,194.0818 143.9291,197.1701 138.5347,204.9449 137.7165,200.6181 131.5039,203.059 131.5039,202.3865 125.2913,201.2907 119.0787,202.2836 119.0787,201.1599 112.8661,202.5172 112.8661,200.5537 106.6535,203.1235 106.6535,199.7656 100.4409,203.9116 100.4409,205.7347 94.2283,210.3676 94.2283,208.4574 88.0157,207.645 88.0157,204.3347 94.2283,199.3424 94.2283,203.1161 100.4409,200.5611"/>
 <polyline points="199.8425,209.919 206.0551,206.1834 206.0551,209.6153 212.2677,206.4871 212.2677,208.5259 218.4803,207.5764 224.6929,208.2072 224.6929,209.5713 230.9055,206.531 230.9055,209.495 237.1181,206.6073 237.1181,208.831 243.3307,207.2714 243.3307,208.6018 249.5433,207.5006 249.5433,208.6169 255.7559,207.4855 255.7559,214.5431 261.9685,213.9844 261.9685,216.1031 269.9087,211.1575 272.6661,204.9449 274.3937,208.5318 280.6063,207.5705 280.6063,208.4315 286.8189,207.6709 286.8189,210.2487 294.0902,204.9449 298.1854,198.7323 299.2441,198.2266 293.0315,193.0254 293.0315,193.721 286.8189,197.5309 286.8189,194.8356 280.6063,196.4164 280.6063,192.9733 274.3937,198.2787 274.3937,193.869 268.1811,197.383 268.1811,199.549 261.9685,204.1282 256.3092,211.1575 254.0492,211.1575 251.25,217.3701 249.5433,214.5801 243.3307,213.9474 243.3307,212.1379 237.1181,216.3897 234.754,217.3701 233.2696,223.5827 230.9055,220.2165 224.6929,220.7362 224.6929,219.9192 218.4803,221.0413 212.2677,219.844 206.0551,221.1087 206.0551,219.6933 199.8425,221.2595 199.8425,222.2788 193.6299,218.6739 193.6299,221.031 187.4173,219.5427 181.2047,221.4101 180.1659,217.3701 176.031,223.5827 174.9921,218.8264 168.7795,222.1263 168.7795,218.2119 162.5669,222.7408 162.5669,218.8467 156.3543,222.1061 156.3543,219.6035 150.1417,221.3493 150.1417,219.9497 143.9291,221.003 143.9291,219.7675 137.7165,221.1852 137.7165,221.7303 131.5039,219.2225 125.2913,220.8411 119.0787,220.1117 119.0787,221.2088 112.8661,219.744 112.8661,221.1084 106.6535,219.8443 106.6535,219.1997 100.4409,221.7531 99.3168,217.3701 95.3525,223.5827 94.2283,219.6048 88.0157,221.348 88.0157,219.7309 81.8031,221.2218 81.8031,219.1955 75.5906,221.7572 75.5906,214.5991 81.8031,213.9285 81.8031,213.4103 88.0157,215.1173 88.0157,213.681 94.2283,214.8466 94.2283,213.6005 100.4409,214.9271 100.4409,214.0044 106.6535,214.6973 112.8661,213.8303 112.8661,215.5379 119.0787,212.9897 119.0787,213.4318 125.2913,215.0958 125.2913,217.2848 131.5039,211.2427 131.5039,210.5113 137.7165,205.591 137.7165,206.2105 143.9291,2

In [73]:
sk.vpype('linesort')
        
savepath = Path(savedir).joinpath(filename).as_posix()
sk.save(savepath)